In [5]:
import tensorflow as tf
import numpy as np
import random
import time
import argparse
from math import sqrt

try:
    from config import config
    from DilatedRNN import DilatedRNNWavefunction
    from utils import Fullyconnected_localenergies, Fullyconnected_diagonal_matrixelements
    from vca import vca_solver
    print("Successfully imported local modules.")
except ImportError:
    print("Local .py files not found. Please ensure they are in the same directory or paste the classes below.")

tf.compat.v1.disable_eager_execution()

/home/hanyuuu/miniconda3/envs/vca/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hanyuuu/miniconda3/envs/vca/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hanyuuu/miniconda3/envs/vca/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/hanyuuu/miniconda3/envs/vca/lib/python3.7/s

Successfully imported local modules.


# Section 1: Ising Model as Energy Minimization

## 1.1 What is the Ising Model?
The Ising model was originally developed in statistical physics to describe the **ferromagnetism** of solid materials. It assumes that a system consists of a collection of **spins** arranged on a lattice, where each spin can only take one of two discrete values:
* $s_i = +1$ (Spin Up)
* $s_i = -1$ (Spin Down)

In this implementation, the spins are mapped to binary values $s \in \{0, 1\}$ to be compatible with neural network processing.

## 1.2 Mathematical Nature & The Hamiltonian
The **Hamiltonian** is a function in physics that represents the total energy of a system. For a fully connected Ising model, the energy function $E(\mathbf{s})$ is defined as:

$$E(\mathbf{s}) = -\sum_{i < j} J_{ij} \sigma_i \sigma_j - \sum_i h_i \sigma_i$$

* **$J_{ij}$ (Interaction Term)**: Defines the coupling strength between spins $i$ and $j$.
    * If $J_{ij} > 0$: Spins tend to align in the same direction (Ferromagnetic).
    * If $J_{ij} < 0$: Spins tend to align in opposite directions (Anti-ferromagnetic).
* **$h_i$ (External Field)**: Represents the influence of an external magnetic field or bias on an individual spin. In this code, a transverse field `Bx` is introduced to add non-diagonal perturbations.
* **Code Example**: In `utils.py`, the `Jz` matrix stores all interaction weights, and the function `Fullyconnected_diagonal_matrixelements` calculates the classical energy based on these weights.



In [ ]:
def Fullyconnected_diagonal_matrixelements(Jz, samples):
    numsamples = samples.shape[0]
    N = samples.shape[1]
    energies = np.zeros((numsamples), dtype = np.float64)

    for i in range(N-1):
      values = np.expand_dims(samples[:,i], axis = -1)+samples[:,i+1:]
      valuesT = np.copy(values)
      valuesT[values==2] = +1 #If both spins are up
      valuesT[values==0] = +1 #If both spins are down
      valuesT[values==1] = -1 #If they are opposite

      energies += np.sum(valuesT*(-Jz[i,i+1:]), axis = 1)

    return energies

## 1.3 Problem Type: Combinatorial Optimization
From a computer science perspective, the Ising model is a classic **Combinatorial Optimization Problem**:
* **Goal**: To find a specific arrangement among $2^N$ possible discrete configurations that minimizes the total energy of the system.
* **Applications**: Many famous NP-hard problems, such as the **Max-Cut Problem**, **Traveling Salesperson Problem (TSP)**, or **Graph Coloring**, can be exactly mapped onto the energy minimization of an Ising model.

## 1.4 The Essence of NP-hardness
Finding the lowest energy state (the **Ground State**) of an Ising model is extremely difficult for the following reasons:

1.  **Exponential State Space**: For $N$ spins, there are $2^N$ possible states. Brute-force search becomes impossible as $N$ grows.
2.  **Frustration**: When the distribution of $J_{ij}$ is complex (as in Spin Glass models), the system cannot satisfy all coupling terms simultaneously leading towards numerous equivalent ground states.
3.  **Complex Energy Landscape**: Frustration leads to a highly non-convex energy landscape filled with numerous **Local Minima**.
4.  **Mathematical Bottleneck**: Due to these high energy barriers, traditional optimization algorithms easily get stuck in local optima, failing to find the global optimum. This is the core of its NP-hardness.

# Section 2:Simulated Annealing as a Classical Heuristic

## 2.1 What is Simulated Annealing
**Simulated Annealing (SA)** is a classical stochastic optimization algorithm designed to find low-energy (or high-quality) solutions in large, discrete, and non-convex search spaces.

The key idea of simulated annealing is simple:

> Instead of greedily accepting only better solutions, SA occasionally accepts worse solutions in the early stage of the search, allowing it to escape local minima. As the search progresses, this randomness is gradually reduced, and the algorithm becomes more conservative.

This behavior is inspired by the physical annealing process, where a material is slowly cooled to reach a low-energy crystalline state. However, in optimization, simulated annealing should be understood primarily as a **search heuristic**, not a physical simulation.

## 2.2 Simulated Annealing as a Search Process

Simulated annealing can be understood as a **randomized search process** over the solution space.

At each step, the algorithm starts from a current solution and proposes a small random change. If the new solution improves the objective, it is accepted. If it is worse, it may still be accepted, especially at the early stage of the search.

This randomness is important: it allows the algorithm to escape local minima and explore different regions of the solution space.

The temperature parameter controls how random the search is:
- At high temperature, the algorithm explores more freely.
- At low temperature, the algorithm becomes more conservative and focuses on improving solutions.

By gradually lowering the temperature, simulated annealing transitions from exploration to refinement.

**Key idea:**  
Simulated annealing is not a deterministic optimization method, but a guided random search that balances exploration and exploitation.

### 2.2.1 Boltzmann Distribution (Intuition)

The Boltzmann distribution is a simple way to express the idea that
**solutions with lower energy are more likely than solutions with higher energy**.

It introduces a temperature parameter that controls this preference:
- At high temperature, many solutions are treated similarly.
- At low temperature, low-energy solutions are strongly preferred.

You do not need to compute this distribution explicitly to use simulated annealing.

---

### Why Is It Mentioned in Simulated Annealing?

In simulated annealing, the Boltzmann distribution is used only to **explain** the acceptance rule.

- Better solutions are always accepted.
- Worse solutions may still be accepted, especially when the temperature is high.

As the temperature decreases, the algorithm becomes less likely to accept worse solutions and gradually focuses on improving the solution.

**Key idea:**  
The Boltzmann distribution helps us understand *why* simulated annealing behaves the way it does, but the algorithm itself never computes this distribution.


# Section 3: From Simulated Annealing to Learing

## 3.1 Limitations of Simulated Annealing
Simulated annealing is a well-established classical heuristic, but it exhibits several limitations when applied to large-scale and highly structured optimization problems.

The search behavior of simulated annealing is **manually designed**.
Key components such as neighborhood moves, acceptance rules, and temperature schedules are specified a priori and remain fixed, limiting the algorithm’s ability to adapt to instance-specific structure.

Moreover, simulated annealing does not **accumulate or reuse experience**.
Each run is independent, and information obtained during previous searches is not leveraged to improve future performance.

In rugged energy landscapes such as spin glass systems, the resulting random walk may spend a substantial amount of time exploring suboptimal regions before reaching low-energy configurations.

These limitations motivate learning-based approaches, which aim not to eliminate randomness, but to **learn heuristics that more effectively guide the search process**.

## 3.2 Why learning methods make sense


Compared to classical simulated annealing, learning-based methods offer several practical advantages when applied to large-scale NP-hard optimization problems. Simulated annealing relies on fixed, hand-designed components such as neighborhood moves and temperature schedules, which remain unchanged across different problem instances. In contrast, learning-based methods can adapt the search strategy by extracting patterns from data or repeated search experience, allowing the heuristic to better align with the underlying structure of the problem. 

Moreover, while simulated annealing treats each run independently, learning-based approaches can amortize computational cost across multiple instances by reusing learned knowledge, enabling faster generation of high-quality candidate solutions at inference time. This shift from instance-specific random exploration to experience-driven guidance is particularly beneficial in complex energy landscapes, where naive random walks may suffer from slow exploration and high computational cost. Importantly, learning-based methods do not remove randomness or circumvent the NP-hard nature of the problem, but they can significantly improve practical efficiency by guiding randomized search toward more promising regions of the solution space.

# Section 4: VNA as Example

Here is the link to VNA Github: https://github.com/VectorInstitute/VariationalNeuralAnnealing/tree/main?tab=readme-ov-file

## 4.1 Motivation

Simulated annealing solves optimization problems by approximately sampling a sequence of Boltzmann distributions, but its effectiveness is often limited by slow sampling dynamics in rugged or glassy energy landscapes. In practical, finite-time runs, the system deviates from equilibrium, leading to suboptimal solutions.

Variational Neural Annealing addresses this limitation by replacing the exact Boltzmann distribution with a flexible, parameterized model that can be efficiently sampled. Instead of annealing the physical system itself, VNA anneals a learnable probability distribution whose parameters are optimized using variational principles and energy feedback. Modern autoregressive neural networks provide an ideal parameterization, as they allow exact and fast sampling even when representing complex, highly structured landscapes. This shift from trajectory-based sampling to distribution-based learning enables more efficient exploration of low-energy configurations under limited computational budgets.

## 4.2 Algorithm Logic

This section provides a step-by-step walkthrough of a learning-based annealing method for Ising optimization.  
Each conceptual step is explicitly linked to a corresponding function in the implementation, allowing readers to connect algorithmic ideas with concrete code components.



### 1. Load an Ising Instance

**Relevant functions:**  
- `config.read_graph(graph_path)`  
- `config.__init__()`

We begin by loading an Ising problem instance from file. The graph structure and coupling strengths are encoded in a dense interaction matrix , which defines the Ising Hamiltonian. This step specifies the optimization problem and is independent of the solver.

In [9]:
# Initialize the configuration with a specific Ising instance
instance_path = "dataset/EA/EA_10x10/10x10_uniform_seed1.txt" 
cfg = config(instance_path, seed=42)

# Verify the system scale and problem dimensions
print(f"System Size (N): {cfg.N}") # Number of spins (sites)
print(f"J-Matrix Shape: {cfg.Jz.shape}") # Should be an (N x N) matrix

# Physical Consistency Check: The interaction J_ij must equal J_ji (Symmetry)
# In physics, the coupling between spin i and spin j is mutual.
is_symmetric = (cfg.Jz == cfg.Jz.T).all()
print(f"Is the J-matrix symmetric? {is_symmetric}")

# Inspect a sub-section of the interaction matrix
# This represents a local 'snapshot' of the couplings between the first 5 spins.
print("\nLocal Sampling of J-Matrix (Top 5x5):")
print(cfg.Jz[:5, :5])

System Size (N): 100
J-Matrix Shape: (100, 100)
Is the J-matrix symmetric? True

Local Sampling of J-Matrix (Top 5x5):
[[ 0.         -0.16595599  0.          0.          0.        ]
 [-0.16595599  0.          0.60148914  0.          0.        ]
 [ 0.          0.60148914  0.          0.97772218  0.        ]
 [ 0.          0.          0.97772218  0.         -0.79533114]
 [ 0.          0.          0.         -0.79533114  0.        ]]


### 2. Define the Neural Sampler

**Relevant class:**  
- `DilatedRNNWavefunction`

The neural sampler parameterizes a probability distribution over spin configurations. We use an autoregressive recurrent neural network to model the joint distribution of spins, enabling exact and efficient sampling without Markov chain dynamics. The network serves as a learnable heuristic that guides the randomized search process.

In [16]:
# Each RNN layer uses the same number of hidden units.
# cfg.num_layers controls how many dilated layers are used.
# cfg.num_units controls the capacity of each RNN cell.
units = [cfg.num_units] * cfg.num_layers
# Instantiate the autoregressive neural sampler
rnn_sampler = DilatedRNNWavefunction(
    systemsize=cfg.N,                 # Number of spins in the Ising model
    units=units,                      # Hidden units per RNN layer
    layers=cfg.num_layers,            # Number of dilated RNN layers
    activation=cfg.activation_function,
    seed=cfg.seed
)

print("Number of spins (N):", cfg.N)
print("Number of RNN layers:", cfg.num_layers)
print("Hidden units per layer:", cfg.num_units)
print("RNN cells per layer:", len(rnn_sampler.rnn[0]))
print("Total number of RNN cells:", cfg.num_layers * cfg.N)
#Each layer contains one RNN cell per spin position.  
#This design allows the model to capture both short-range and long-range dependencies between spins through dilated connections.

Number of spins (N): 100
Number of RNN layers: 10
Hidden units per layer: 40
RNN cells per layer: 100
Total number of RNN cells: 1000


### 3. Sampling Spin Configurations

**Relevant method:**  
- `DilatedRNNWavefunction.sample(numsamples, inputdim)`

At each iteration, the model generates a batch of complete spin configurations by sampling from the learned distribution. Each sample corresponds to a candidate solution to the Ising optimization problem. Randomness is preserved, but the sampling is biased by the learned model parameters.

---

In [21]:
# Number of configurations to sample at each iteration
num_samples = cfg.numsamples

# IMPORTANT:
# The following call builds the sampling operations in the
# TensorFlow 1.x computation graph.
# `samples` and `log_probs` are symbolic Tensors, not concrete values yet.
samples, log_probs = rnn_sampler.sample(
    numsamples=num_samples,
    inputdim=2
)

print("Symbolic sample tensor shape:", samples.shape)
print("Symbolic log-probability tensor shape:", log_probs.shape)

# In TensorFlow 1.x, tensors must be explicitly evaluated
# inside a Session to obtain NumPy arrays.
with tf.compat.v1.Session(graph=rnn_sampler.graph) as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    
    # Run the sampling operation
    samples_val, log_probs_val = sess.run([samples, log_probs])

# At this point, `samples_val` and `log_probs_val` are NumPy arrays
print("Sample shape (NumPy):", samples_val.shape)
print("First 5 sampled configurations:")
print(samples_val[:5])


Sample shape: (50, 100)
Log-probability shape: (50,)


2026-01-25 23:12:58.291598: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2026-01-25 23:12:58.315903: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3686395000 Hz
2026-01-25 23:12:58.319744: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x12d952e0 executing computations on platform Host. Devices:
2026-01-25 23:12:58.319784: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>


Sample shape: (50, 100)
First 5 sampled configurations:
[[1 1 0 0 0 1 1 0 1 0 0 1 0 0 1 1 1 1 0 1 1 0 0 0 0 1 1 1 0 0 0 1 1 0 1 0
  0 1 1 0 1 1 1 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 1 0 1 0
  1 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 0 1 1 1 0 0 1 1 0 0 0]
 [1 0 0 1 1 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 1 1 0 0 0 1 1 1 1 1 1 1 0 0
  0 1 0 1 0 0 0 0 1 1 1 0 0 1 0 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 0 0 0 1 1 0
  0 0 0 0 1 1 1 1 0 0 0 1 0 0 0 0 1 1 0 1 1 1 1 0 1 0 1 1]
 [0 1 0 1 1 0 0 0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 1 0 1 0 1 1 1 1 0 0 0 0 0 1
  0 0 0 1 0 1 1 0 0 1 0 1 0 0 1 1 0 1 1 1 1 1 1 1 0 0 0 0 0 1 0 0 1 0 1 0
  1 1 0 1 1 0 1 1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 1 0 0 1 1]
 [1 0 1 0 1 0 0 1 1 0 1 0 0 0 1 0 0 1 1 1 0 1 0 1 1 0 1 1 0 0 1 1 1 1 0 0
  1 0 1 1 1 1 0 0 1 1 0 1 1 0 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 0 0 0
  0 1 1 0 0 1 0 1 0 1 0 0 0 1 1 0 1 1 1 1 0 0 1 0 0 1 1 1]
 [1 1 1 0 0 1 0 1 1 0 1 1 0 0 1 1 0 0 0 0 1 0 1 0 1 1 1 0 0 1 0 0 1 0 1 1
  1 0 1 0 0 1 0 1 1 1 0 1 0 1 1 1 1 0 1 0 


### 4. Energy Evaluation

**Relevant functions:**  
- `Fullyconnected_diagonal_matrixelements(Jz, samples)`  
- `Fullyconnected_localenergies(...)` (optional)

The quality of sampled configurations is evaluated using the Ising Hamiltonian. Energy values provide the only feedback signal used to guide learning. Lower-energy configurations correspond to better approximate solutions.

---

In [22]:
# Jz encodes the pairwise couplings of the Ising Hamiltonian
Jz = cfg.Jz

# Compute classical Ising energies for each sampled configuration
energies = Fullyconnected_diagonal_matrixelements(Jz, samples_val)

print("Energy array shape:", energies.shape)
print("Mean energy:", energies.mean())
print("Minimum energy:", energies.min())

Energy array shape: (50,)
Mean energy: 0.73026605526684
Minimum energy: -25.023755028118764


### 5. Training with Annealing

**Relevant function:**  
- `vca_solver(config)`

Model parameters are updated using energy feedback under a temperature-controlled objective. A warmup phase stabilizes learning, followed by an annealing schedule that gradually shifts the focus from exploration to exploitation. Unlike classical simulated annealing, annealing here regulates the learning objective rather than the sampling dynamics.

Training begins with a **warmup phase**, during which the model stabilizes before annealing is applied.  
This is followed by an **annealing phase**, where the temperature is gradually reduced. At higher temperatures, the learning objective encourages exploration across diverse configurations. As the temperature decreases, updates increasingly focus on concentrating probability mass around low-energy regions.

A key distinction from classical simulated annealing is that annealing in VCA does not control the sampling dynamics. Instead, it regulates the learning objective, acting as a curriculum that guides how the sampling distribution evolves over time.

### 6. Final Sampling for Benchmarking

**Relevant code block:**  
- Final sampling section in `vca_solver`

After the annealing process completes, the trained model is used to generate a large number of spin configurations efficiently. The minimum and average energies of these samples are reported as benchmark metrics, reflecting the practical performance of the learned heuristic.

## 4.3 Execution Cell
To run the whole solver in this Notebook, use the following code block. Ensure you have the problem instance file (e.g., `10x10_uniform_seed1.txt`) in your working directory.

In [24]:
# 1. Path to your problem instance
instance_path = "dataset/EA/EA_10x10/10x10_uniform_seed1.txt" 
seed = 0

# 2. Initialize configuration
vca_config = config(instance_path, seed)

# 3. Run the solver
# This will output the annealing progress, energy (E), and free energy (F)
mean_energies, min_energies = vca_solver(vca_config)

print(f"\nFinal Results:")
print(f"Minimum Energy Found: {min_energies}")
print(f"Mean Energy: {mean_energies}")



Number of spins = 100
Initial_temperature = 2
Seed =  0

Number of annealing steps = 256
Number of training steps = 2280
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
mean(E): 0.2452989836958769, mean(F): -136.89834102581816, var(E): 62.51400790693402, var(F): 69.76507423649147, #samples 50, #Training step 0
Temperature:  2
Magnetic field:  0
Elapsed time is = 17.47144317626953  seconds



mean(E): 1.2486735244024691, mean(F): -134.76014703919012, var(E): 76.68254870717614, var(F): 78.83708195921278, #samples 50, #Training step 5
Temperature:  2
Magnetic field:  0
Elapsed time is = 17.91935443878174  seconds



mean(E): -1.0110286029135487, mean(F): -136.43445324233556, var(E): 68.52024181833383, var(F): 77.46636476047603, #samples 50, #Training step 10
Temperature:  2
Magnetic field:  0
Elapsed time is = 18.420081853866577  seconds



mean(E): -0.1795473178707069, mean(F): -134.7838739850909, var(E): 53.5500

mean(E): -12.47350770752053, mean(F): -143.28263083415663, var(E): 70.97582459934353, var(F): 42.93329518483672, #samples 50, #Training step 165
Temperature:  2
Magnetic field:  0
Elapsed time is = 32.53054141998291  seconds



mean(E): -12.878928459115162, mean(F): -143.92304419745577, var(E): 70.90078054222083, var(F): 61.52694263159505, #samples 50, #Training step 170
Temperature:  2
Magnetic field:  0
Elapsed time is = 32.99649214744568  seconds



mean(E): -12.70000871598274, mean(F): -143.46748939967924, var(E): 73.44176213075895, var(F): 40.553959605767034, #samples 50, #Training step 175
Temperature:  2
Magnetic field:  0
Elapsed time is = 33.45564889907837  seconds



mean(E): -13.637884329704253, mean(F): -144.8448738204191, var(E): 63.01975654962345, var(F): 40.93181700319794, #samples 50, #Training step 180
Temperature:  2
Magnetic field:  0
Elapsed time is = 33.905094385147095  seconds



mean(E): -13.337016361162739, mean(F): -144.86314397851953, var(E): 39.34325077820164

mean(E): -25.68859091425289, mean(F): -150.29138430886564, var(E): 64.1507915376759, var(F): 14.948559363035145, #samples 50, #Training step 345
Temperature:  2
Magnetic field:  0
Elapsed time is = 52.5973916053772  seconds



mean(E): -25.707486099830806, mean(F): -151.4564326953707, var(E): 50.853389444245, var(F): 16.752829261897, #samples 50, #Training step 350
Temperature:  2
Magnetic field:  0
Elapsed time is = 53.055408000946045  seconds



mean(E): -27.741629552596724, mean(F): -151.56098739381244, var(E): 50.46467863925624, var(F): 14.656674810117305, #samples 50, #Training step 355
Temperature:  2
Magnetic field:  0
Elapsed time is = 53.50150918960571  seconds



mean(E): -27.321182229001742, mean(F): -151.32659495197163, var(E): 68.55556591665348, var(F): 14.694818140057894, #samples 50, #Training step 360
Temperature:  2
Magnetic field:  0
Elapsed time is = 53.95057201385498  seconds



mean(E): -28.085310523732083, mean(F): -152.691403606729, var(E): 70.79084681086864, var

mean(E): -29.64693301238982, mean(F): -153.81007057068612, var(E): 50.20988296510462, var(F): 4.882321141191389, #samples 50, #Training step 525
Temperature:  2
Magnetic field:  0
Elapsed time is = 68.93689680099487  seconds



mean(E): -31.264593062728625, mean(F): -153.67842647581728, var(E): 86.96815080449095, var(F): 6.949115222213249, #samples 50, #Training step 530
Temperature:  2
Magnetic field:  0
Elapsed time is = 69.37546348571777  seconds



mean(E): -29.720468148118094, mean(F): -153.56327717010262, var(E): 49.331561055816245, var(F): 6.040980963004614, #samples 50, #Training step 535
Temperature:  2
Magnetic field:  0
Elapsed time is = 69.81979894638062  seconds



mean(E): -30.518962519352762, mean(F): -153.85041827811506, var(E): 45.76657518032393, var(F): 6.002844353596451, #samples 50, #Training step 540
Temperature:  2
Magnetic field:  0
Elapsed time is = 70.2692461013794  seconds



mean(E): -29.859217023997584, mean(F): -154.1561430024969, var(E): 38.78946969070382,

mean(E): -28.983682319913665, mean(F): -154.0674670477729, var(E): 63.16880049428109, var(F): 2.9205101933127455, #samples 50, #Training step 705
Temperature:  2
Magnetic field:  0
Elapsed time is = 88.73787689208984  seconds



mean(E): -30.215039214892826, mean(F): -154.23633777713005, var(E): 60.47717099948474, var(F): 3.6843872385570777, #samples 50, #Training step 710
Temperature:  2
Magnetic field:  0
Elapsed time is = 89.18490934371948  seconds



mean(E): -30.550405681568193, mean(F): -154.37187688250458, var(E): 54.66458933716223, var(F): 2.4553614976155065, #samples 50, #Training step 715
Temperature:  2
Magnetic field:  0
Elapsed time is = 89.62243247032166  seconds



mean(E): -31.95394185747086, mean(F): -154.21101080194111, var(E): 66.21409948930054, var(F): 3.9789814937630634, #samples 50, #Training step 720
Temperature:  2
Magnetic field:  0
Elapsed time is = 90.0683708190918  seconds



mean(E): -31.493215029419243, mean(F): -154.96511939139862, var(E): 69.175105565506

...


Annealing step: 105.0/256
mean(E): -46.45842341052881, mean(F): -107.83418697783858, var(E): 42.80611047829491, var(F): 3.6217236277704217, #samples 50, #Training step 1525
Temperature:  1.1796875
Magnetic field:  0.0
Elapsed time is = 169.75002717971802  seconds




Annealing step: 106.0/256
mean(E): -47.63437205118482, mean(F): -107.11756703108762, var(E): 49.16806706807182, var(F): 3.4147942357765233, #samples 50, #Training step 1530
Temperature:  1.171875
Magnetic field:  0.0
Elapsed time is = 170.19883346557617  seconds




Annealing step: 107.0/256
mean(E): -48.233467240159655, mean(F): -107.18148692198702, var(E): 31.856516771493062, var(F): 3.209650756615576, #samples 50, #Training step 1535
Temperature:  1.1640625
Magnetic field:  0.0
Elapsed time is = 170.6339237689972  seconds




Annealing step: 108.0/256
mean(E): -48.40007532009746, mean(F): -106.35315005727138, var(E): 26.02781963776129, var(F): 3.5323890610030895, #samples 50, #Training step 1540
Temperature:  1.15625
M


Annealing step: 136.0/256
mean(E): -56.580954608746744, mean(F): -96.2504277626041, var(E): 31.88462249136774, var(F): 2.0941650823773976, #samples 50, #Training step 1680
Temperature:  0.9375
Magnetic field:  0.0
Elapsed time is = 183.730961561203  seconds




Annealing step: 137.0/256
mean(E): -56.46882062306191, mean(F): -95.90118667491606, var(E): 24.706629533970332, var(F): 2.7580185250760962, #samples 50, #Training step 1685
Temperature:  0.9296875
Magnetic field:  0.0
Elapsed time is = 184.22690558433533  seconds




Annealing step: 138.0/256
mean(E): -56.356513211734665, mean(F): -95.1272899288511, var(E): 37.62799458730973, var(F): 3.5303458639295457, #samples 50, #Training step 1690
Temperature:  0.921875
Magnetic field:  0.0
Elapsed time is = 184.66038918495178  seconds




Annealing step: 139.0/256
mean(E): -56.559098098912685, mean(F): -94.83492387439289, var(E): 38.79184890933468, var(F): 2.207561269215863, #samples 50, #Training step 1695
Temperature:  0.9140625
Magneti


Annealing step: 167.0/256
mean(E): -64.75128804451333, mean(F): -87.44527582187531, var(E): 17.36931674264888, var(F): 0.8365653314011746, #samples 50, #Training step 1835
Temperature:  0.6953125
Magnetic field:  0.0
Elapsed time is = 201.26814723014832  seconds




Annealing step: 168.0/256
mean(E): -66.13041429772235, mean(F): -87.37627082994817, var(E): 20.88258584525256, var(F): 1.2868845901462873, #samples 50, #Training step 1840
Temperature:  0.6875
Magnetic field:  0.0
Elapsed time is = 201.7117636203766  seconds




Annealing step: 169.0/256
mean(E): -65.4626078254925, mean(F): -86.81282583424547, var(E): 16.103498252234534, var(F): 1.2570673158099281, #samples 50, #Training step 1845
Temperature:  0.6796875
Magnetic field:  0.0
Elapsed time is = 202.15049982070923  seconds




Annealing step: 170.0/256
mean(E): -66.25339556344649, mean(F): -86.74705311492693, var(E): 11.905594086394373, var(F): 0.8191528517225616, #samples 50, #Training step 1850
Temperature:  0.671875
Magnet


Annealing step: 198.0/256
mean(E): -72.43608745494029, mean(F): -81.58602976250589, var(E): 7.81584946867928, var(F): 0.7216021587790057, #samples 50, #Training step 1990
Temperature:  0.453125
Magnetic field:  0.0
Elapsed time is = 215.26263213157654  seconds




Annealing step: 199.0/256
mean(E): -72.97472006337657, mean(F): -81.49994794375621, var(E): 7.733425653915316, var(F): 0.8130604794634158, #samples 50, #Training step 1995
Temperature:  0.4453125
Magnetic field:  0.0
Elapsed time is = 215.70778894424438  seconds




Annealing step: 200.0/256
mean(E): -73.74211541247388, mean(F): -81.60810959559218, var(E): 4.631724647607327, var(F): 0.3091626162858519, #samples 50, #Training step 2000
Temperature:  0.4375
Magnetic field:  0.0
Elapsed time is = 216.1620969772339  seconds




Annealing step: 201.0/256
mean(E): -73.80935327788096, mean(F): -81.33816991608836, var(E): 5.703599071403248, var(F): 0.39876098607889204, #samples 50, #Training step 2005
Temperature:  0.4296875
Magneti


Annealing step: 229.0/256
mean(E): -77.48055070332231, mean(F): -79.0189654470622, var(E): 1.6238477618167644, var(F): 0.45249811066333934, #samples 50, #Training step 2145
Temperature:  0.2109375
Magnetic field:  0.0
Elapsed time is = 232.66618371009827  seconds




Annealing step: 230.0/256
mean(E): -77.58958097863731, mean(F): -79.01167009833195, var(E): 0.9933991905147913, var(F): 0.26051956893295947, #samples 50, #Training step 2150
Temperature:  0.203125
Magnetic field:  0.0
Elapsed time is = 233.11851978302002  seconds




Annealing step: 231.0/256
mean(E): -77.74388676107172, mean(F): -79.03293422897265, var(E): 0.8540491310583337, var(F): 0.07770768484988802, #samples 50, #Training step 2155
Temperature:  0.1953125
Magnetic field:  0.0
Elapsed time is = 233.56041479110718  seconds




Annealing step: 232.0/256
mean(E): -77.56199206314079, mean(F): -78.91862399004066, var(E): 1.0463593021305637, var(F): 0.1524221532013812, #samples 50, #Training step 2160
Temperature:  0.1875
